In [1]:
import torch

criterion = torch.nn.CrossEntropyLoss()
Y = torch.LongTensor([2,0,1])

Y_pred1 = torch.Tensor([[0.1,0.2,0.9],
                       [1.1,0.1,0.2],
                       [0.2,2.1,0.1]])
Y_pred2 = torch.Tensor([[0.8,0.2,0.3],
                       [0.2,0.3,0.5],
                       [0.2,0.2,0.5]])

l1 = criterion(Y_pred1, Y)
l2 = criterion(Y_pred2, Y)

print("Batch Loss1 = ",l1.data,"\n Batch Loss2 = ", l2.data)

Batch Loss1 =  tensor(0.4966) 
 Batch Loss2 =  tensor(1.2389)


In [24]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

batch_size = 64
#将图像的0-255变换为0-1的张量，convert the PIL Image to Tensor，Normalize(均值,标准差)
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])

train_dataset = datasets.MNIST(root = '../dataset/mnist/', train = True, download = True, transform = transform)
train_loader = DataLoader(train_dataset, shuffle = True, batch_size = batch_size)
test_dataset = datasets.MNIST(root = '../dataset/mnist/', train = False, download = True, transform = transform)
test_loader = DataLoader(train_dataset, shuffle = True, batch_size = batch_size)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784,512)
        self.l2 = torch.nn.Linear(512,256)
        self.l3 = torch.nn.Linear(256,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,10)
        
    def forward(self,x):
        x = x.view(-1,784)#-1表示自动获取mini_batch
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
    
model = Net()

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum = 0.5) #momentum冲量

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()        
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss: %.3f' % (epoch + 1, batch_idx + 1,running_loss/300))
            running_loss = 0.0
            
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item() #张量比较后求和
        print('accuracy on test set: %d %%' % (100 * correct / total))
        
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()
        

[1,  300] loss: 2.184
[1,  600] loss: 0.860
[1,  900] loss: 0.418
accuracy on test set: 89 %
[2,  300] loss: 0.293
[2,  600] loss: 0.261
[2,  900] loss: 0.226
accuracy on test set: 94 %
[3,  300] loss: 0.182
[3,  600] loss: 0.162
[3,  900] loss: 0.146
accuracy on test set: 96 %
[4,  300] loss: 0.124
[4,  600] loss: 0.122
[4,  900] loss: 0.108
accuracy on test set: 97 %
[5,  300] loss: 0.097
[5,  600] loss: 0.083
[5,  900] loss: 0.087
accuracy on test set: 98 %
[6,  300] loss: 0.071
[6,  600] loss: 0.071
[6,  900] loss: 0.072
accuracy on test set: 98 %
[7,  300] loss: 0.055
[7,  600] loss: 0.060
[7,  900] loss: 0.053
accuracy on test set: 98 %
[8,  300] loss: 0.046
[8,  600] loss: 0.042
[8,  900] loss: 0.051
accuracy on test set: 98 %
[9,  300] loss: 0.039
[9,  600] loss: 0.036
[9,  900] loss: 0.036
accuracy on test set: 99 %
[10,  300] loss: 0.029
[10,  600] loss: 0.026
[10,  900] loss: 0.034
accuracy on test set: 99 %
